In [2]:
import pickle
import warnings
from collections import Counter
from random import randint

import nltk
import numpy as np
import pandas as pd
import tensorflow.keras.backend as K
from gensim.models import FastText
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow import keras
from tqdm import tqdm

warnings.filterwarnings('ignore')
nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/slfdstrctd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/slfdstrctd/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Считаем препроцессные данные из прошлого ноутбучка

In [3]:
interactions_df = pd.read_csv('../data/interactions_processed.csv')
users_df = pd.read_csv('../data/users_processed.csv')
items_df = pd.read_csv('../data/items_processed.csv')

In [4]:
users_df.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,M,True
1,962099,age_18_24,income_20_40,M,False
2,1047345,age_45_54,income_40_60,F,False
3,721985,age_45_54,income_20_40,F,False
4,704055,age_35_44,income_60_90,F,False


In [5]:
items_df = items_df.rename(columns={'id': 'item_id'})

In [6]:
items_df.head()

,item_id,content_type,title,title_orig,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat
0,10711,film,поговори с ней,Hable con ella,"драмы, зарубежные, детективы, мелодрамы",испания,False,16.0,unknown,педро альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...",2000-2010
1,2508,film,голые перцы,Search Party,"зарубежные, приключения, комедии",сша,False,16.0,unknown,скот армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...",2010-2020
2,10716,film,тактическая сила,Tactical Force,"криминал, зарубежные, триллеры, боевики, комедии",канада,False,16.0,unknown,адам п. калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...",2010-2020
3,7868,film,45 лет,45 Years,"драмы, зарубежные, мелодрамы",великобритания,False,16.0,unknown,эндрю хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...",2010-2020
4,16268,film,все решает мгновение,None,"драмы, спорт, советские, мелодрамы",ссср,False,12.0,ленфильм,виктор садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...",1970-1980


In [7]:
items_df.shape

(15963, 14)

In [8]:
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72
1,699317,1659,2021-05-29,8317,100
2,656683,7107,2021-05-09,10,0
3,864613,7638,2021-07-05,14483,100
4,964868,9506,2021-04-30,6725,100


## Готовим фичи пользователей

Посмотрим, какие фичи в датасете фильмов являются категориальными и закодируем их с помощью one-hot encoding.

In [9]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами 
# из датафрейма с взаимодействиями
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом, 
    # который мы получили на предыдущем шаге
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()


,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,age_age_unknown,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,income_income_unknown,sex_F,sex_M,sex_sex_unknown,kids_flg_False,kids_flg_True
0,973171,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
1,962099,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
2,1047345,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0
3,721985,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
4,704055,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0


## Готовим фичи айтемов

Кодируем их точно так же - one-hot'ом.

In [10]:
items_df.head()

,item_id,content_type,title,title_orig,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat
0,10711,film,поговори с ней,Hable con ella,"драмы, зарубежные, детективы, мелодрамы",испания,False,16.0,unknown,педро альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...",2000-2010
1,2508,film,голые перцы,Search Party,"зарубежные, приключения, комедии",сша,False,16.0,unknown,скот армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...",2010-2020
2,10716,film,тактическая сила,Tactical Force,"криминал, зарубежные, триллеры, боевики, комедии",канада,False,16.0,unknown,адам п. калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...",2010-2020
3,7868,film,45 лет,45 Years,"драмы, зарубежные, мелодрамы",великобритания,False,16.0,unknown,эндрю хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...",2010-2020
4,16268,film,все решает мгновение,None,"драмы, спорт, советские, мелодрамы",ссср,False,12.0,ленфильм,виктор садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...",1970-1980


In [11]:
item_cat_feats = ['content_type', 'release_year_cat',
                  'for_kids', 'age_rating',
                  'studios', 'countries', 'directors']

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
    ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1)

items_ohe_df.head()

,item_id,content_type_film,content_type_series,release_year_cat_1920-1930,release_year_cat_1930-1940,release_year_cat_1940-1950,release_year_cat_1950-1960,release_year_cat_1960-1970,release_year_cat_1970-1980,release_year_cat_1980-1990,...,directors_ярив хоровиц,directors_ярон зильберман,directors_ярополк лапшин,directors_ярослав лупий,"directors_ярроу чейни, скотт моужер",directors_ясина сезар,directors_ясуоми умэцу,"directors_ёдзи фукуяма, ацуко фукусима, николас де креси, синъитиро ватанабэ, сёдзи кавамори",directors_ёлкин туйчиев,directors_ён сан-хо
0,10711,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2508,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10716,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7868,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16268,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
items_ohe_df.shape

(15963, 8589)

## Сделаем матрицу взаимодействий

In [13]:
interactions_df.item_id.value_counts()

10440    202457
15297    193123
9728     132865
13865    122119
4151      91167
          ...  
8076          1
8954          1
15664         1
818           1
10542         1
Name: item_id, Length: 15706, dtype: int64

В датасете взаимодействий есть непопулярные фильмы и малоактивные пользователи. Кроме того, в таблице взаимодействий есть события с низким качеством взаимодействия - когда юзер начал смотреть фильм, но вскоре после начала просмотра выключил.

Отфильтруем такие события*, малоактивных юзеров и непопулярные фильмы.

Можете не фильтровать такие события, тогда у вас будет больше негативных примеров.

In [14]:
interactions_df.user_id.value_counts()

416206     1341
1010539     764
555233      685
11526       676
409259      625
           ... 
45493         1
615194        1
96848         1
425823        1
697262        1
Name: user_id, Length: 962179, dtype: int64

In [15]:
print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 10 процентов
interactions_df = interactions_df[interactions_df.watched_pct > 10]

# соберем всех пользователей, которые посмотрели 
# больше 10 фильмов (можете выбрать другой порог)
valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 10 пользователей
valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 962179
N items before: 15706

N users after: 79515
N items after: 6901


После фильтрации может получиться так, что некоторые айтемы/юзеры есть в датасете взаимодействий, но при этом они отсутствуют в датасетах айтемов/юзеров или наоборот. Поэтому найдем id айтемов и id юзеров, которые есть во всех датасетах и оставим только их.

In [16]:
common_users = set(interactions_df.user_id.unique()).intersection(
    set(users_ohe_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(
    set(items_ohe_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

65974
6901


Соберем взаимодействия в матрицу user*item так, чтобы в строках этой матрицы были user_id, в столбцах - item_id, а на пересечениях строк и столбцов - единица, если пользователь взаимодействовал с айтемом и ноль, если нет.

Такую матрицу удобно собирать в numpy array, однако нужно помнить, что numpy array индексируется порядковыми индексами, а нам же удобнее использовать item_id и user_id.

Создадим некие внутренние индексы для user_id и item_id - uid и iid. Для этого просто соберем все user_id и item_id и пронумеруем их по порядку.

In [17]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


,user_id,item_id,last_watch_dt,total_dur,watched_pct,uid,iid
0,176549,9506,2021-05-11,4250,72,10616,3944
1,699317,1659,2021-05-29,8317,100,42131,675
6,1016458,354,2021-08-14,1672,25,61024,139
7,884009,693,2021-08-04,703,14,53150,279
14,5324,8437,2021-04-18,6598,92,310,3485


Отнормируем матрицу взаимодействий

In [18]:
interactions_vec = np.zeros((interactions_df.uid.nunique(),
                             interactions_df.iid.nunique()))

for user_id, item_id, watched_pct in zip(interactions_df.uid, interactions_df.iid, interactions_df.watched_pct):
    interactions_vec[user_id, item_id] += watched_pct / 100 # учтём взаимодействие - процент просмотра

res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]

In [19]:
print(interactions_df.item_id.nunique())
print(items_ohe_df.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_ohe_df.user_id.nunique())

print(set(items_ohe_df.item_id.unique()) - set(interactions_df.item_id.unique()))

6897
6901
65974
65974
{11805, 9788, 11501, 1734}


Для того, чтобы можно было удобно превратить iid/uid в item_id/user_id и наоборот соберем словари 

{iid: item_id}, {uid: user_id} и {item_id: iid}, {user_id: uid}.

In [23]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()[
    "item_id"]
item_id_to_iid = \
    interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()[
    "user_id"]
user_id_to_uid = \
    interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

И проиндексируем датасеты users_ohe_df и items_ohe_df по внутренним айди:

In [24]:
items_ohe_df['item_id'].head()

8      9853
10     3526
15    15076
16     2904
20     2635
Name: item_id, dtype: int64

In [25]:
items_ohe_df.shape

(6901, 8589)

In [26]:
for x in items_ohe_df['item_id']:
    if x not in item_id_to_iid:
        print(x)

11805
11501
9788
1734


In [27]:
items_ohe_df["iid"] = items_ohe_df["item_id"].apply(
    lambda x: item_id_to_iid[x] if x in item_id_to_iid else None)
items_ohe_df = items_ohe_df.set_index("iid")

users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid[x])
users_ohe_df = users_ohe_df.set_index("uid")

## Текстовые признаки

In [28]:
items_df.shape

(15963, 14)

In [29]:
# items_common_df = items_df[items_df.item_id.isin(common_items)]

In [88]:
item_text_feats = ['title', 'directors',
                  'actors', 'description',
                  'keywords']

In [89]:
stop_words = set(stopwords.words('russian'))

In [90]:
for feature in item_text_feats:
    items_df[f'{feature}_processed'] = items_df[feature].str.lower()
    items_df[f'{feature}_processed'] = items_df[f'{feature}_processed'].apply(word_tokenize)
    items_df[f'{feature}_processed'] = items_df[f'{feature}_processed'].apply(lambda x: [word for word in x if word.isalnum() and word not in stop_words]).apply(lambda x : " ".join(x))

In [91]:
item_text_feats_proc = ['title_processed', 'directors_processed',
                  'actors_processed', 'description_processed',
                  'keywords_processed']

In [92]:
items_df['combined_text'] = items_df[item_text_feats_proc].apply(lambda x: " ".join(x), axis=1)
items_df.shape

(15963, 20)

In [93]:
tokenized_text = items_df['combined_text'] #.tolist()

embedding_size = 100
min_word_count = 1

model = FastText(sentences=tokenized_text, vector_size=100, min_count=min_word_count)

In [127]:
embeddings = items_df['combined_text'].apply(lambda x: model.wv[x])

In [130]:
embeddings_df = pd.DataFrame(embeddings.tolist(), index=embeddings.index)

In [131]:
embeddings_df['item_id'] = items_df['item_id']

In [144]:
embeddings_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,item_id
iid,,,,,,,,,,,,,,,,,,,,,
NaN,-0.000175,-0.000069,0.000032,0.000183,0.000021,-1.822629e-05,0.000039,-0.000104,-0.000149,-0.000189,...,-0.000032,0.000021,0.000039,0.000054,-0.000025,0.000281,-0.000171,-0.000009,-0.000032,10711
NaN,-0.000052,-0.000106,0.000055,0.000164,-0.000090,1.520634e-05,-0.000163,-0.000021,-0.000117,0.000007,...,-0.000044,0.000023,-0.000055,-0.000086,0.000034,0.000065,-0.000038,-0.000085,0.000001,2508
NaN,-0.000133,0.000019,-0.000038,0.000140,0.000100,-1.250927e-04,-0.000129,0.000010,-0.000084,0.000031,...,-0.000090,0.000075,-0.000189,0.000050,-0.000093,0.000038,-0.000087,-0.000034,-0.000110,10716
NaN,-0.000026,0.000048,0.000041,0.000163,0.000060,-7.728289e-07,-0.000066,0.000006,-0.000166,-0.000032,...,-0.000220,0.000027,0.000025,-0.000051,-0.000069,0.000062,0.000002,0.000033,0.000054,7868
NaN,-0.000252,0.000015,0.000098,0.000096,-0.000083,3.796032e-05,-0.000068,-0.000150,-0.000119,-0.000105,...,-0.000002,-0.000124,0.000028,-0.000018,-0.000162,0.000145,-0.000058,0.000027,0.000090,16268


In [137]:
embeddings_df["iid"] = embeddings_df["item_id"].apply(
    lambda x: item_id_to_iid[x] if x in item_id_to_iid else None)
embeddings_df = embeddings_df.set_index("iid")

In [146]:
items_ohe_df = items_ohe_df.merge(embeddings_df)

In [147]:
items_ohe_df

,item_id,content_type_film,content_type_series,release_year_cat_1920-1930,release_year_cat_1930-1940,release_year_cat_1940-1950,release_year_cat_1950-1960,release_year_cat_1960-1970,release_year_cat_1970-1980,release_year_cat_1980-1990,...,90,91,92,93,94,95,96,97,98,99
0,9853,1,0,0,0,0,0,0,0,0,...,-0.000051,0.000126,0.000053,0.000002,0.000011,0.000040,0.000129,-0.000021,-0.000025,0.000144
1,3526,1,0,0,0,0,0,0,0,0,...,-0.000076,-0.000105,-0.000023,-0.000057,0.000064,-0.000066,0.000178,0.000026,-0.000027,0.000108
2,15076,1,0,0,0,0,0,0,0,0,...,0.000073,0.000055,-0.000065,-0.000021,0.000066,-0.000109,-0.000028,0.000042,0.000055,0.000094
3,2904,1,0,0,0,0,0,0,0,0,...,0.000149,-0.000086,-0.000110,0.000207,-0.000099,-0.000077,0.000155,-0.000012,0.000100,-0.000121
4,2635,1,0,0,0,0,0,0,0,0,...,-0.000150,0.000004,-0.000100,0.000028,0.000052,0.000055,0.000086,-0.000041,0.000020,0.000137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6896,15610,0,1,0,0,0,0,0,0,0,...,0.000077,0.000093,0.000037,-0.000011,-0.000255,-0.000239,-0.000028,0.000154,-0.000157,0.000035
6897,6443,0,1,0,0,0,0,0,0,0,...,0.000032,-0.000126,0.000091,0.000180,0.000098,-0.000039,0.000154,-0.000002,-0.000092,0.000009
6898,2367,0,1,0,0,0,0,0,0,0,...,-0.000175,-0.000022,-0.000128,-0.000043,-0.000071,0.000007,0.000068,-0.000363,0.000080,0.000230
6899,10632,0,1,0,0,0,0,0,0,0,...,-0.000083,0.000151,0.000005,0.000012,-0.000049,-0.000237,0.000204,0.000081,0.000168,0.000126


In [177]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    # будем ожидать, что на вход функции прилетит три сконкатенированных 
    # вектора - вектор юзера и два вектора айтема
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims:n_dims * 2]
    negative = y_pred[:, n_dims * 2:n_dims * 3]

    # считаем расстояния от вектора юзера до вектора хорошего айтема
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # и до плохого
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # считаем лосс
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0)  # возвращаем ноль, если лосс отрицательный

    return loss

Попробуйте другие лоссы, например, BPR Triplet loss

## Генератор и семплирование

Сделаем простой генератор. Он будет брать рандромного юзера, и два разных айтема - хороший пример и плохой:
- хорошим примером будет тот айтем, который был взят из датасета взаимодействий в соответствии с распределением просмотренных айтемов для этого юзера;
- а плохим айтемом будет просто любой другой _случайный айтем_*

In [149]:
def generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            # берем рандомный uid
            uid_i = randint(0, interactions.shape[0] - 1)
            # id хорошего айтема
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i])
            # id плохого айтема
            neg_i = np.random.choice(range(interactions.shape[1]))
            # фичи юзера
            uid_meta.append(users.iloc[uid_i])
            # вектор айтемов, с которыми юзер взаимодействовал
            uid_interaction.append(interactions_vec[uid_i])
            # фичи хорошего айтема
            pos.append(items.iloc[pos_i])
            # фичи плохого айтема
            neg.append(items.iloc[neg_i])

        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [
            np.array(uid_meta), np.array(uid_interaction)]


In [150]:
# инициализируем генератор
gen = generator(items=items_ohe_df.drop(["item_id"], axis=1),
                users=users_ohe_df.drop(["user_id"], axis=1),
                interactions=interactions_vec)

ret = next(gen)

print(f"вектор фичей юзера: {ret[0][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[0][1].shape}")
print(f"вектор 'хорошего' айтема: {ret[0][2].shape}")
print(f"вектор 'плохого' айтема: {ret[0][3].shape}")
print()
print(f"вектор фичей юзера: {ret[1][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[1][1].shape}")

вектор фичей юзера: (1024, 19)
вектор взаимодействий юзера с айтемами: (1024, 6897)
вектор 'хорошего' айтема: (1024, 8688)
вектор 'плохого' айтема: (1024, 8688)

вектор фичей юзера: (1024, 19)
вектор взаимодействий юзера с айтемами: (1024, 6897)


### Сделайте генаратор, который будет использовать информацию о качестве взаимодействия юзеров с айтемами для более репрезентативного сэмплирования

In [151]:
N_FACTORS = 128

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1],)
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1],)

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1],)

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (8688,)
USER_META_MODEL_SHAPE: (19,)
USER_INTERACTION_MODEL_SHAPE: (6897,)


In [152]:
def item_model(n_factors=N_FACTORS):
    # входной слой
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

    # полносвязный слой
    layer_1 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp)

    # делаем residual connection - складываем два слоя, 
    # чтобы градиенты не затухали во время обучения
    layer_2 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1)

    add = keras.layers.Add()([layer_1, layer_2])

    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(add)

    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    # входной слой для вектора фичей юзера (из users_ohe_df)
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    # входной слой для вектора просмотров (из iteractions_vec)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

    # полносвязный слой
    layer_1_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                      kernel_regularizer=keras.regularizers.l2(1e-6),
                                      activity_regularizer=keras.regularizers.l2(l2=1e-6))(
        inp_meta)

    layer_1_interaction = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                             kernel_regularizer=keras.regularizers.l2(1e-6),
                                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(
        inp_interaction)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                      kernel_regularizer=keras.regularizers.l2(1e-6),
                                      activity_regularizer=keras.regularizers.l2(l2=1e-6))(
        layer_1_meta)

    add = keras.layers.Add()([layer_1_meta, layer_2_meta])

    # конкатенируем вектор фичей с вектором просмотров
    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])

    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(
        concat_meta_interaction)

    return keras.models.Model([inp_meta, inp_interaction], out)


# инициализируем модели юзера и айтема
i2v = item_model()
u2v = user_model()

# вход для вектора фичей юзера (из users_ohe_df)
ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
# вход для вектора просмотра юзера (из interactions_vec)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

# вход для вектора "хорошего" айтема
pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
# вход для вектора "плохого" айтема
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# получаем вектор юзера
ancor = u2v([ancor_meta_in, ancor_interaction_in])
# получаем вектор "хорошего" айтема
pos = i2v(pos_in)
# получаем вектор "плохого" айтема
neg = i2v(neg_in)

# конкатенируем полученные векторы
res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

# собираем модель
model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

2023-12-12 23:59:53.147958: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-12 23:59:53.166785: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-12 23:59:53.166856: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-12 23:59:53.168548: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-12 23:59:53.168617: I external/local_xla/xla/stream_executor

In [153]:
model_name = 'recsys_resnet_linear'

# логируем процесс обучения в тензорборд
t_board = keras.callbacks.TensorBoard(log_dir=f'runs/{model_name}')

# уменьшаем learning_rate, если лосс долго не уменьшается (в течение двух эпох)
decay = keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, factor=0.8, verbose=1)

# сохраняем модель после каждой эпохи, если лосс уменьшился
check = keras.callbacks.ModelCheckpoint(filepath=model_name + '/epoch{epoch}-{loss:.2f}.h5',
                                        monitor="loss")


In [154]:
# компилируем модель, используем оптимайзер Adam и triplet loss
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

In [155]:
# модель айтема
item_model().summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 8688)]               0         []                            
                                                                                                  
 dense_7 (Dense)             (None, 128)                  1112064   ['input_8[0][0]']             
                                                                                                  
 dense_8 (Dense)             (None, 128)                  16384     ['dense_7[0][0]']             
                                                                                                  
 add_2 (Add)                 (None, 128)                  0         ['dense_7[0][0]',             
                                                                     'dense_8[0][0]']       

In [156]:
# модель юзера
user_model().summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                                  
 dense_10 (Dense)            (None, 128)                  2432      ['input_9[0][0]']             
                                                                                                  
 dense_12 (Dense)            (None, 128)                  16384     ['dense_10[0][0]']            
                                                                                                  
 input_10 (InputLayer)       [(None, 6897)]               0         []                            
                                                                                            

In [157]:
# общая модель
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 6897)]               0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 8688)]               0         []                            
                                                                                                  
 input_7 (InputLayer)        [(None, 8688)]               0         []                            
                                                                                            

In [158]:
# начинаем обучение, не забывая дропнуть столбцы item_id и user_id 
# из датафреймов при инициализации генератора.

# batch_size можно (и лучше) поставить побольше, если вы не органичены в ресурсах

model.fit(generator(items=items_ohe_df.drop(["item_id"], axis=1),
                    users=users_ohe_df.drop(["user_id"], axis=1),
                    interactions=interactions_vec,
                    batch_size=64),
          steps_per_epoch=100,
          epochs=30,
          initial_epoch=0,
          callbacks=[decay, t_board, check]
          )

Epoch 1/30


2023-12-13 00:00:02.422001: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fe683489400 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-13 00:00:02.422038: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2023-12-13 00:00:02.433012: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-13 00:00:02.456793: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1702414802.524505   29368 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


100/100 [==============================] - 46s 445ms/step - loss: 0.3054 - lr: 0.0010
Epoch 2/30
100/100 [==============================] - 45s 452ms/step - loss: 0.2172 - lr: 0.0010
Epoch 3/30
100/100 [==============================] - 46s 467ms/step - loss: 0.1989 - lr: 0.0010
Epoch 4/30
100/100 [==============================] - 46s 467ms/step - loss: 0.1818 - lr: 0.0010
Epoch 5/30
100/100 [==============================] - 46s 460ms/step - loss: 0.1724 - lr: 0.0010
Epoch 6/30
100/100 [==============================] - 45s 455ms/step - loss: 0.1733 - lr: 0.0010
Epoch 7/30
100/100 [==============================] - 45s 450ms/step - loss: 0.1632 - lr: 0.0010
Epoch 8/30
100/100 [==============================] - 45s 450ms/step - loss: 0.1604 - lr: 0.0010
Epoch 9/30
100/100 [==============================] - 45s 451ms/step - loss: 0.1580 - lr: 0.0010
Epoch 10/30
100/100 [==============================] - 44s 448ms/step - loss: 0.1497 - lr: 0.0010
Epoch 11/30
100/100 [===================

In [159]:
with open('dssm_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [172]:
items_ohe_df.to_csv('items_ohe_df.csv', index=False)  

In [160]:
# берем рандомного юзера
rand_uid = np.random.choice(list(users_ohe_df.index))

# получаем фичи юзера и вектор его просмотров айтемов
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid]
user_interaction_vec = interactions_vec[rand_uid]

In [161]:
# берем рандомный айтем
rand_iid = np.random.choice(list(items_ohe_df.index))
# получаем фичи айтема
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[int(rand_iid)]

In [162]:
# получаем вектор юзера
user_vec = u2v.predict([np.array(user_meta_feats).reshape(1, -1),
                        np.array(user_interaction_vec).reshape(1, -1)])

# и вектор айтема
item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))

# считаем расстояние между вектором юзера и вектором айтема
from sklearn.metrics.pairwise import euclidean_distances as ED

ED(user_vec, item_vec)

1/1 [==============================] - 0s 37ms/step


array([[1.0587355]], dtype=float32)

In [173]:
# получаем фичи всех айтемов
items_feats = items_ohe_df.drop(["item_id"], axis=1).to_numpy()
# получаем векторы всех айтемов
items_vecs = i2v.predict(items_feats)

# считаем расстояния
dists = ED(user_vec, items_vecs)

216/216 [==============================] - 0s 1ms/step


In [175]:
dists.shape

(1, 6901)

In [164]:
def recommend(user_id, items_vecs, topn=10):
    uid = user_id_to_uid[user_id]
    user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[uid]
    user_interaction_vec = interactions_vec[uid]
    user_vec = u2v.predict(
        [np.array(user_meta_feats).reshape(1, -1), np.array(user_interaction_vec).reshape(1, -1)],
        verbose=False,
    )
    dists = ED(user_vec, items_vecs)
    topn_iids = np.argsort(dists, axis=1)[:, :topn]
    topn_iids_item = [iid_to_item_id[iid] for iid in topn_iids.reshape(-1)]
    return topn_iids_item

In [167]:
interactions_df.user_id

0           176549
1           699317
6          1016458
7           884009
14            5324
            ...   
5476218     840031
5476224     663351
5476226     435089
5476239     610017
5476249     384202
Name: user_id, Length: 1463641, dtype: int64

In [181]:
recs = {}


for user_id in tqdm(set(common_users)):
    user_recs = recommend(user_id, items_vecs)
    recs.update({user_id: user_recs})

100%|██████████| 65974/65974 [1:24:47<00:00, 12.97it/s]


In [182]:
with open('../saved_models/dssm_offline.pkl', 'wb') as f:
    pickle.dump(recs, f)